# 🎯 리셀러 자동 판단 시스템\n\n**사용법: 위에서부터 순서대로 ▶️ 버튼을 클릭하세요!**

In [ ]:
# 1️⃣ 첫 번째: 파일 업로드 (▶️ 클릭 후 파일 선택)\nfrom google.colab import files\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport re\nfrom datetime import datetime\n\n# 한글 폰트 설정\n!apt-get -qq install fonts-nanum\n!fc-cache -fv\nplt.rcParams['font.family'] = 'NanumGothic'\nplt.rcParams['axes.unicode_minus'] = False\n\nprint(\"📁 엑셀 파일을 업로드하세요!\")\nprint(\"   파일명: erp_order_form.xlsx\")\nuploaded = files.upload()\n\nif 'erp_order_form.xlsx' in uploaded:\n    print(\"✅ 파일 업로드 성공!\")\nelse:\n    print(\"❌ 파일명을 확인하세요: erp_order_form.xlsx\")

In [ ]:
# 2️⃣ 두 번째: 데이터 읽기 (▶️ 클릭)\ntry:\n    # 오전/오후 시트 통합\n    df_morning = pd.read_excel('erp_order_form.xlsx', sheet_name='오전')\n    df_afternoon = pd.read_excel('erp_order_form.xlsx', sheet_name='오후')\n    df = pd.concat([df_morning, df_afternoon], ignore_index=True)\n    \n    print(f\"✅ 데이터 읽기 완료!\")\n    print(f\"📊 총 {len(df)}건의 주문 데이터\")\n    print(f\"👥 고객 수: {df['주문자명'].nunique()}명\")\nexcept Exception as e:\n    print(f\"❌ 에러: {e}\")

In [ ]:
# 3️⃣ 세 번째: 리셀러 분석 (▶️ 클릭)\n\ndef detect_business_address(address):\n    if pd.isna(address):\n        return 0\n    address_str = str(address)\n    business_keywords = ['사무실', '오피스', '빌딩', '타워', '센터', '상가', '층',\n                        '호실', '공장', '창고', '물류', '유통', '산업']\n    for keyword in business_keywords:\n        if keyword in address_str:\n            return 1\n    return 0\n\ndef analyze_reseller_patterns(df):\n    df['주문일자'] = pd.to_datetime(df['주문일자'], errors='coerce')\n    \n    analysis = df.groupby('주문자명').agg({\n        '주문수량': ['sum', 'mean', 'std'],\n        '총매출액': 'sum',\n        '주문번호(쇼핑몰)': 'count',\n        '상품명': ['nunique', lambda x: ', '.join(x.value_counts().head(3).index)],\n        '주소': lambda x: x.mode()[0] if len(x) > 0 else '',\n        '주문일자': ['min', 'max', lambda x: (x.max() - x.min()).days if len(x) > 1 else 0]\n    }).round(2)\n    \n    analysis.columns = ['총구매수량', '평균구매수량', '수량편차', '총매출액', '주문횟수',\n                        '구매제품종류', '주력상품TOP3', '대표주소',\n                        '첫구매일', '최근구매일', '활동기간(일)']\n    \n    analysis['사업장주소'] = analysis['대표주소'].apply(detect_business_address)\n    analysis['평균구매주기'] = (analysis['활동기간(일)'] / analysis['주문횟수']).replace([float('inf'), -float('inf')], 0).round(1)\n    \n    return analysis\n\ndef calculate_reseller_score(stats):\n    scores = pd.DataFrame(index=stats.index)\n    \n    scores['대량구매'] = stats['평균구매수량'].apply(\n        lambda x: 30 if x >= 5 else 20 if x >= 3 else 10 if x >= 2 else 0\n    )\n    scores['정기구매'] = stats.apply(\n        lambda x: 25 if (x['주문횟수'] >= 5 and x['활동기간(일)'] >= 30) else\n                  15 if (x['주문횟수'] >= 3 and x['활동기간(일)'] >= 14) else\n                  5 if x['주문횟수'] >= 2 else 0,\n        axis=1\n    )\n    scores['사업장'] = stats['사업장주소'] * 20\n    scores['다양성'] = stats['구매제품종류'].apply(\n        lambda x: 15 if x >= 5 else 10 if x >= 3 else 5 if x >= 2 else 0\n    )\n    scores['구매규모'] = stats['총매출액'].apply(\n        lambda x: 10 if x >= 200000 else 7 if x >= 100000 else 3 if x >= 50000 else 0\n    )\n    \n    scores['리셀러점수'] = scores.sum(axis=1)\n    scores['판정'] = scores['리셀러점수'].apply(\n        lambda x: '🔴 확실한 리셀러' if x >= 70 else\n                  '🟠 리셀러 가능성 높음' if x >= 50 else\n                  '🟡 잠재 리셀러' if x >= 30 else\n                  '🟢 일반 고객'\n    )\n    scores['등급'] = scores['리셀러점수'].apply(\n        lambda x: 'A' if x >= 70 else 'B' if x >= 50 else 'C' if x >= 30 else 'D'\n    )\n    \n    final = pd.concat([stats, scores], axis=1)\n    return final\n\n# 분석 실행\nprint(\"🔍 리셀러 분석 중...\")\ncustomer_analysis = analyze_reseller_patterns(df)\nfinal_results = calculate_reseller_score(customer_analysis)\n\n# 통계\ntotal = len(final_results)\na_grade = len(final_results[final_results['등급'] == 'A'])\nb_grade = len(final_results[final_results['등급'] == 'B'])\nc_grade = len(final_results[final_results['등급'] == 'C'])\n\nprint(f\"\\n✅ 분석 완료!\")\nprint(f\"📊 리셀러 분포:\")\nprint(f\"🔴 A등급 (확실한 리셀러): {a_grade}명\")\nprint(f\"🟠 B등급 (가능성 높음): {b_grade}명\")\nprint(f\"🟡 C등급 (잠재 리셀러): {c_grade}명\")\nprint(f\"🟢 D등급 (일반 고객): {total-a_grade-b_grade-c_grade}명\")

In [ ]:
# 4️⃣ 네 번째: 상위 리셀러 보기 (▶️ 클릭)\n\nprint(\"🔥 상위 리셀러 후보 TOP 10\")\nprint(\"=\" * 60)\n\ntop_resellers = final_results.nlargest(10, '리셀러점수')\n\nfor rank, (idx, row) in enumerate(top_resellers.iterrows(), 1):\n    print(f\"\\n{rank}위. {idx}\")\n    print(f\"   📊 등급: {row['판정']}\")\n    print(f\"   🎯 점수: {row['리셀러점수']:.0f}점\")\n    print(f\"   💰 매출: {row['총매출액']:,.0f}원\")\n    print(f\"   📦 주문: {row['주문횟수']}회, 평균 {row['평균구매수량']:.1f}개\")

In [ ]:
# 5️⃣ 다섯 번째: 엑셀 다운로드 (▶️ 클릭)\n\n# 엑셀 파일 생성\nexport_data = final_results.sort_values('리셀러점수', ascending=False)\nexport_columns = [\n    '등급', '판정', '리셀러점수',\n    '총구매수량', '평균구매수량', '주문횟수',\n    '총매출액', '구매제품종류',\n    '사업장주소', '활동기간(일)', '평균구매주기',\n    '주력상품TOP3'\n]\n\n# 엑셀 저장\nexport_data[export_columns].to_excel('리셀러_분석_결과.xlsx')\n\nprint(\"💾 엑셀 파일 생성 완료!\")\nprint(\"📥 아래 다운로드 버튼을 클릭하세요!\")\n\n# 다운로드\nfiles.download('리셀러_분석_결과.xlsx')